In [1]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import datetime as dt

Data needed for configuration of the Option's pricings

In [39]:
# Definition and calculation of the basket's key inputs
"""
S0: array of spot prices for each asset. Calculated input
weights: array of basket weights. Assumed input
K: strike price. Assumed input
T: time to maturity (in years). Calculated input
rf_rate: risk-free rate (annual, decimal), used in the Monte carlo simulation. Calculated input
sigma: array of volatilities for each asset. Calculated input
corr_matrix: correlation matrix between assets. Calculated input
N: number of paths (different scenarios). Assumed input
"""

# Define the basket's tickers and weights
tickers = ['BHP.AX', 'CSL.AX', 'WDS.AX', 'MQG.AX']
weights = np.array([0.1, 0.35, 0.15, 0.4])

# Extracting closing prices as of Friday, 16 May 2025 from yfinance
data = yf.download(tickers, start='2024-05-16', end='2025-05-17')['Close']
closing_prices = data.loc['2025-05-16'].values

# Calculate weighted average (basket spot price, S0)
S0 = np.dot(weights, closing_prices)
print("Basket spot price S0 as of 16 May 2025:", S0)
print("Closing prices on 16 May 2025:", closing_prices)

#Download the latest 13-week treasury bill rate for the risk-free rate, r, on 16 May 2025
rf_data = yf.download('^IRX', start='2025-05-10', end='2025-05-17')['Close']
# Drop missing values
rf_data = rf_data.dropna()
if rf_data.empty:
    raise ValueError("No risk-free rate data available for the selected period.")
else:
    rf_rate = rf_data.iloc[-1] / 100  # Use last available, convert from % to decimal
    rf_rate_value = rf_rate.values[0] if hasattr(rf_rate, "values") else float(rf_rate)
    r = rf_rate_value
    print(f"Risk-free rate as of {rf_data.index[-1].date()}: {rf_rate_value:.4%}")

# Calculate daily log returns for volatility and correlation estimation
returns = np.log(data / data.shift(1)).dropna()

# Annualized volatilities
sigma = returns.std() * np.sqrt(252)
sigma = sigma.values

# Correlation matrix
corr_matrix = returns.corr().values

# Calculate basket volatility
cov_matrix = np.outer(sigma, sigma) * corr_matrix
sigma_basket = np.sqrt(weights @ cov_matrix @ weights)

print("Basket's volatilities (sigma):", sigma_basket)


[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed

Basket spot price S0 as of 16 May 2025: 128.4795036315918
Closing prices on 16 May 2025: [ 39.72000122 241.82000732 207.3500061   21.92000008]
Risk-free rate as of 2025-05-16: 4.2370%
Basket's volatilities (sigma): 0.17620663202111472


In [36]:
def binomial_european_call(S0, K, T, sigma, N):
    """Calculate European call option price using binomial model"""
    # Calculate time step
    dt = T/N
    print(f"dt: {dt}")
    
    # Calculate up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    print(f"u: {u}")
    d = 1/u
    print(f"d: {d}")
    
    # Calculate risk-neutral probability
    p = 1 / (u + 1)
    print(f"p: {p}")
    
    # Initialize stock price tree
    stock_tree = np.zeros((N+1, N+1))
    
    # Populate the stock price tree through time
    for i in range(N+1):
        for j in range(i+1):
            stock_tree[i, j] = S0 * (u ** j) * (d ** (i - j))
    
    # Initialize option value tree
    option_tree = np.zeros((N+1, N+1))
    
    # Calculate call option payoffs at expiration
    for j in range(N+1):
        option_tree[N, j] = max(0, stock_tree[N, j] - K)
    
    # Work backwards discounting and reflecting risk-neutral probabilities to calculate option value at each node
    for i in range(N-1, -1, -1):
        for j in range(i+1):
            option_tree[i, j] = (p * option_tree[i+1, j+1] + (1-p) * option_tree[i+1, j])
    
    return option_tree[0, 0], stock_tree, option_tree

In [38]:
K = 75
valuation_date = dt. datetime(2025, 5, 16)
expiry_date = dt.datetime(2025, 7, 17)
T = (expiry_date - valuation_date).days / 365
N = 100

price, stock_tree, option_tree = binomial_european_call(S0, K, T, sigma_basket, N)
print(f"European call basket option price: {price:.2f} AUD")
print("Time until expiry (T):", T)

dt: 0.0016986301369863012
u: 1.00728869196405
d: 0.9927640486563606
p: 0.4981844435249326
European call basket option price: 53.48 AUD
Time until expiry (T): 0.16986301369863013


In [31]:
# Monte Carlo simulation for European basket call option
def monte_carlo_basket_call(S0, weights, K, T, r, sigma, corr_matrix, N=100):
    n_assets = len(S0)
    # Cholesky decomposition for correlated random numbers
    L = np.linalg.cholesky(corr_matrix)
    # Simulate correlated standard normals
    Z = np.random.normal(size=(N, n_assets))
    correlated_Z = Z @ L.T
    # Simulate asset prices at maturity
    drift = (r - 0.5 * sigma ** 2) * T
    diffusion = sigma * np.sqrt(T)
    S_T = S0 * np.exp(drift + diffusion * correlated_Z)
    # Calculate basket value at maturity
    basket_T = S_T @ weights
    # Calculate payoff
    payoff = np.maximum(basket_T - K, 0)
    # Discounted expected payoff
    price_mc = np.exp(-r * T) * np.mean(payoff)
    #Time step
    dt = T/N
    print(f"dt: {dt}")
    return price_mc

In [34]:
K = 75
valuation_date = dt. datetime(2025, 5, 16)
expiry_date = dt.datetime(2025, 7, 17)
T = (expiry_date - valuation_date).days / 365

price_mc = monte_carlo_basket_call(closing_prices, weights, K, T, r, sigma, corr_matrix, N=100)
print(f"Monte Carlo price of the European basket call option: {price_mc:.2f} AUD")

dt: 0.0016986301369863012
Monte Carlo price of the European basket call option: 54.08 AUD
